In [1]:
import numpy as np
import pandas as pd
from numpy import array
import matplotlib.pyplot as plt

from sklearn.feature_selection import f_classif, mutual_info_classif, SelectFdr

from sklearn.feature_selection import GenericUnivariateSelect
from sklearn.svm import LinearSVC, NuSVC
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier 

from scipy.stats import uniform, randint

import numpy as np
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

import xgboost as xgb
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split

from warnings import filterwarnings
filterwarnings('ignore')


In [2]:

def plot_learning_curve(estimator, title, X, y, axes=None, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate 3 plots: the test and training learning curve, the training
    samples vs fit times curve, the fit times vs score curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    axes : array of 3 axes, optional (default=None)
        Axes to use for plotting the curves.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:

          - None, to use the default 5-fold cross-validation,
          - integer, to specify the number of folds.
          - :term:`CV splitter`,
          - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, optional (default=None)
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like, shape (n_ticks,), dtype float or int
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the dtype is float, it is regarded as a
        fraction of the maximum size of the training set (that is determined
        by the selected validation method), i.e. it has to be within (0, 1].
        Otherwise it is interpreted as absolute sizes of the training sets.
        Note that for classification the number of samples usually have to
        be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """
    if axes is None:
        _, axes = plt.subplots(1, 3, figsize=(20, 5))

    axes[0].set_title(title)
    if ylim is not None:
        axes[0].set_ylim(*ylim)
    axes[0].set_xlabel("Training examples")
    axes[0].set_ylabel("Score")

    train_sizes, train_scores, test_scores, fit_times, _ = \
        learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs,
                       train_sizes=train_sizes,
                       return_times=True)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    fit_times_mean = np.mean(fit_times, axis=1)
    fit_times_std = np.std(fit_times, axis=1)

    # Plot learning curve
    axes[0].grid()
    axes[0].fill_between(train_sizes, train_scores_mean - train_scores_std,
                         train_scores_mean + train_scores_std, alpha=0.1,
                         color="r")
    axes[0].fill_between(train_sizes, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1,
                         color="g")
    axes[0].plot(train_sizes, train_scores_mean, 'o-', color="r",
                 label="Training score")
    axes[0].plot(train_sizes, test_scores_mean, 'o-', color="g",
                 label="Cross-validation score")
    axes[0].legend(loc="best")

    # Plot n_samples vs fit_times
    axes[1].grid()
    axes[1].plot(train_sizes, fit_times_mean, 'o-')
    axes[1].fill_between(train_sizes, fit_times_mean - fit_times_std,
                         fit_times_mean + fit_times_std, alpha=0.1)
    axes[1].set_xlabel("Training examples")
    axes[1].set_ylabel("fit_times")
    axes[1].set_title("Scalability of the model")

    # Plot fit_time vs score
    axes[2].grid()
    axes[2].plot(fit_times_mean, test_scores_mean, 'o-')
    axes[2].fill_between(fit_times_mean, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1)
    axes[2].set_xlabel("fit_times")
    axes[2].set_ylabel("Score")
    axes[2].set_title("Performance of the model")

    return plt

In [437]:
def hyper_opt_procedure(model_object,params,cv,perf_df,X,y):


  model = model_object()

  # Initial Model Fit with stock hyper_params

  init_score = np.mean(cross_val_score(model,X,y,cv=cv))

  perf_df = perf_df.append({'Model Iteration': 'Stock Params',
                              'Score':init_score}, ignore_index=True)

  print('Initial Model with Stock Params Score : {}'.format(init_score))

  # Random Search Cross Validation


  search_model = RandomizedSearchCV(model,
                                         param_distributions=params,
                                         random_state=42, 
                                         n_iter=300, 
                                         cv=cv, 
                                         verbose=1, 
                                         n_jobs=1, 
                                         return_train_score=True)

  search_model.fit(X,y)

  best_search_model = search_model.best_estimator_

  best_search_model_score = search_model.best_score_

  perf_df = perf_df.append({'Model Iteration': 'Random HP Search Best Model',
                      'Score':best_search_model_score}, ignore_index=True)

  rd_score_improv = ((best_search_model_score - init_score)/init_score) * 100

  print('Random HP Search Best Model Score : {}'.format(best_search_model_score))

  print('Improvement of {}% relative to the initial model'.format(rd_score_improv))
  
  


  return model, best_search_model
   



class hyper_model:
    
    def __init__(self,model_name,validation_approach,X,y):
        
        self.model_name = model_name
        self.X = X
        self.y = y
        self.validation_approach= validation_approach
        self.best_model = None
        self.best_params = None
        self.all_models_perf = pd.DataFrame(None,columns=['Model Iteration','Score'])
        self.stock_model = None
        self.rd_model = None


    def fit(self):


      if self.validation_approach == 'leave-one-run-out':

        cv = KFold(n_splits=8, shuffle=False)

      elif self.validation_approach == 'leave-one-run-out-shuffled':

        cv = KFold(n_splits=8, shuffle=True, random_state=42)

      elif self.validation_approach == 'leave-one-task-out':

        cv = KFold(n_splits=4, shuffle=False)

      elif self.validation_approach == 'leave-one-task-out-shuffled':

        cv = KFold(n_splits=4, shuffle=True, random_state=42)    


 
        
        # Linear SVM

      if self.model_name == 'linear_svm':

        params = {'C': [0.1, 1, 10]} 

        self.stock_model, self.rd_model = hyper_opt_procedure(model_object=LinearSVC,
                                                                                                 params=params,cv=cv,
                                                                                                 perf_df=self.all_models_perf,
                                                                                                 X=self.X,y=self.y)

         

      # Non Linear SVM


      elif self.model_name == 'non_linear_svm':

        params = { 'kernel': ['rbf','sigmoid']
                       }


        self.stock_model, self.rd_model = hyper_opt_procedure(model_object=NuSVC,
                                                                                                 params=params,cv=cv,
                                                                                                 perf_df=self.all_models_perf,
                                                                                                 X=self.X,y=self.y)

          
      # Random Forest Classifier


      elif self.model_name == 'random_forest_classifier':


        params = {'bootstrap': [True],
                    'max_depth': [60],
                    'max_features': ['auto'],
                    'min_samples_leaf': [4],
                    'min_samples_split': [2, 5],
                    'n_estimators': [1000]
                   }


        self.stock_model, self.rd_model = hyper_opt_procedure(model_object=RandomForestClassifier,
                                                                                                 params=params,cv=cv,
                                                                                                 perf_df=self.all_models_perf,
                                                                                                 X=self.X,y=self.y)
          



      # XG Boost Classifier


      elif self.model_name == 'xg_boost':

        params = {
              "gamma": [0.25,0.5]
              }

        self.stock_model, self.rd_model = hyper_opt_procedure(model_object=xgb.XGBClassifier,
                                                                                                 params=params,cv=cv,
                                                                                                 perf_df=self.all_models_perf,
                                                                                                 X=self.X,y=self.y)  

                
                

In [4]:
X = np.load('../../data/X_pref_lss_shift_quadratic_full.npy')

In [5]:
X.shape

(480, 11457)

In [6]:
y = np.load('../../data/y_pref_lss_shift_quadratic_full.npy')

In [8]:
y.shape

(480, 1)

## Features Selection

### Filter

In [9]:
transformer = GenericUnivariateSelect(mutual_info_classif, mode='k_best', param=550)

In [10]:
X_trans = transformer.fit_transform(X, y)

In [11]:
X_trans.shape

(480, 550)

### Data Preparation

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

## Linear Support Vector Machine

### Stock Model

#### *leave-one-task-out* cross validation

##### Training

In [13]:
hm_linear_svm_to = hyper_model(model_name='linear_svm',
                            validation_approach = 'leave-one-task-out',
                            X=X,
                            y=y)

In [14]:
hm_linear_svm_to.fit()

Initial Model with Stock Params Score : 0.49583333333333335
Fitting 4 folds for each of 5 candidates, totalling 20 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:  2.2min finished


Random HP Search Best Model Score : 0.5166666666666667
Improvement of 4.201680672268915% relative to the initial model


In [15]:
hm_linear_svm_to.all_models_perf

,Model Iteration,Score


##### Test

In [16]:
# leave-one-task-out test
cv = KFold(n_splits=4,shuffle=False)

scores = cross_val_score(estimator=hm_linear_svm_to.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)



In [17]:
scores

array([0.5       , 0.46666667, 0.475     , 0.5       ])

In [18]:
mean_score

0.48541666666666666

In [19]:
# leave-one-task-out-shuffed test
cv = KFold(n_splits=4,shuffle=True,random_state=42)

scores = cross_val_score(estimator=hm_linear_svm_to.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [20]:
scores

array([0.125     , 0.11666667, 0.1       , 0.08333333])

In [21]:
mean_score

0.10625

In [22]:
# leave-one-run-out test
cv = KFold(n_splits=8,shuffle=False)

scores = cross_val_score(estimator=hm_linear_svm_to.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [23]:
scores

array([0.43333333, 0.53333333, 0.45      , 0.43333333, 0.45      ,
       0.53333333, 0.56666667, 0.51666667])

In [24]:
mean_score

0.48958333333333337

In [25]:
# leave-one-run-out-shuffed test
cv = KFold(n_splits=8,shuffle=True,random_state=42)



scores = cross_val_score(estimator=hm_linear_svm_to.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [26]:
scores

array([0.08333333, 0.13333333, 0.03333333, 0.08333333, 0.28333333,
       0.11666667, 0.06666667, 0.01666667])

In [27]:
mean_score

0.10208333333333333

In [28]:
# other metrics + graphs




#### *leave-one-task-out-shuffled* cross validation

##### Training

In [29]:
hm_linear_svm_tos = hyper_model(model_name='linear_svm',
                            validation_approach = 'leave-one-task-out-shuffled',
                            X=X,
                            y=y)

In [30]:
hm_linear_svm_tos.fit()

Initial Model with Stock Params Score : 0.10625
Fitting 4 folds for each of 5 candidates, totalling 20 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:  1.0min finished


Random HP Search Best Model Score : 0.10625
Improvement of 0.0% relative to the initial model


In [31]:
hm_linear_svm_tos.all_models_perf

,Model Iteration,Score


##### Test

In [32]:
# leave-one-task-out test
cv = KFold(n_splits=4,shuffle=False)

scores = cross_val_score(estimator=hm_linear_svm_tos.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)



In [33]:
scores

array([0.54166667, 0.5       , 0.5       , 0.5       ])

In [34]:
mean_score

0.5104166666666666

In [35]:
# leave-one-task-out-shuffed test
cv = KFold(n_splits=4,shuffle=True,random_state=42)

scores = cross_val_score(estimator=hm_linear_svm_tos.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [36]:
scores

array([0.125     , 0.11666667, 0.1       , 0.08333333])

In [37]:
mean_score

0.10625

In [38]:
# leave-one-run-out test
cv = KFold(n_splits=8,shuffle=False)

scores = cross_val_score(estimator=hm_linear_svm_tos.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [39]:
scores

array([0.48333333, 0.46666667, 0.48333333, 0.46666667, 0.58333333,
       0.53333333, 0.46666667, 0.48333333])

In [40]:
mean_score

0.4958333333333333

In [41]:
# leave-one-run-out-shuffed test
cv = KFold(n_splits=8,shuffle=True,random_state=42)



scores = cross_val_score(estimator=hm_linear_svm_tos.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [42]:
scores

array([0.08333333, 0.13333333, 0.03333333, 0.08333333, 0.28333333,
       0.11666667, 0.06666667, 0.01666667])

In [43]:
mean_score

0.10208333333333333

In [44]:
# other metrics + graphs




#### *leave-one-run-out* cross validation

##### Training

In [45]:
hm_linear_svm_ro = hyper_model(model_name='linear_svm',
                            validation_approach = 'leave-one-run-out',
                            X=X,
                            y=y)

In [46]:
hm_linear_svm_ro.fit()

Initial Model with Stock Params Score : 0.51875
Fitting 8 folds for each of 5 candidates, totalling 40 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:  4.7min finished


Random HP Search Best Model Score : 0.51875
Improvement of 0.0% relative to the initial model


In [47]:
hm_linear_svm_ro.all_models_perf

,Model Iteration,Score


##### Test

In [48]:
# leave-one-task-out test
cv = KFold(n_splits=4,shuffle=False)

scores = cross_val_score(estimator=hm_linear_svm_ro.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)



In [49]:
scores

array([0.48333333, 0.44166667, 0.525     , 0.48333333])

In [50]:
mean_score

0.4833333333333334

In [51]:
# leave-one-task-out-shuffed test
cv = KFold(n_splits=4,shuffle=True,random_state=42)

scores = cross_val_score(estimator=hm_linear_svm_ro.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [52]:
scores

array([0.125     , 0.11666667, 0.1       , 0.08333333])

In [53]:
mean_score

0.10625

In [54]:
# leave-one-run-out test
cv = KFold(n_splits=8,shuffle=False)

scores = cross_val_score(estimator=hm_linear_svm_ro.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [55]:
scores

array([0.51666667, 0.55      , 0.51666667, 0.51666667, 0.45      ,
       0.58333333, 0.48333333, 0.5       ])

In [56]:
mean_score

0.5145833333333334

In [57]:
# leave-one-run-out-shuffed test
cv = KFold(n_splits=8,shuffle=True,random_state=42)



scores = cross_val_score(estimator=hm_linear_svm_ro.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [58]:
scores

array([0.08333333, 0.13333333, 0.03333333, 0.08333333, 0.28333333,
       0.11666667, 0.06666667, 0.01666667])

In [59]:
mean_score

0.10208333333333333

In [60]:
# other metrics + graphs




#### *leave-one-run-out-shuffled* cross validation

##### Training

In [61]:
hm_linear_svm_ros = hyper_model(model_name='linear_svm',
                            validation_approach = 'leave-one-run-out-shuffled',
                            X=X,
                            y=y)

In [62]:
hm_linear_svm_ros.fit()

Initial Model with Stock Params Score : 0.1
Fitting 8 folds for each of 5 candidates, totalling 40 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:  4.4min finished


Random HP Search Best Model Score : 0.10416666666666666
Improvement of 4.166666666666652% relative to the initial model


In [63]:
hm_linear_svm_ros.all_models_perf

,Model Iteration,Score


##### Test

In [64]:
# leave-one-task-out test
cv = KFold(n_splits=4,shuffle=False)

scores = cross_val_score(estimator=hm_linear_svm_ros.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)



In [65]:
scores

array([0.48333333, 0.46666667, 0.51666667, 0.50833333])

In [66]:
mean_score

0.49375

In [67]:
# leave-one-task-out-shuffed test
cv = KFold(n_splits=4,shuffle=True,random_state=42)

scores = cross_val_score(estimator=hm_linear_svm_ros.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [68]:
scores

array([0.125     , 0.11666667, 0.1       , 0.08333333])

In [69]:
mean_score

0.10625

In [70]:
# leave-one-run-out test
cv = KFold(n_splits=8,shuffle=False)

scores = cross_val_score(estimator=hm_linear_svm_ros.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [71]:
scores

array([0.51666667, 0.51666667, 0.48333333, 0.48333333, 0.46666667,
       0.43333333, 0.5       , 0.56666667])

In [72]:
mean_score

0.49583333333333335

In [73]:
# leave-one-run-out-shuffed test
cv = KFold(n_splits=8,shuffle=True,random_state=42)



scores = cross_val_score(estimator=hm_linear_svm_ros.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [74]:
scores

array([0.08333333, 0.13333333, 0.03333333, 0.08333333, 0.26666667,
       0.11666667, 0.06666667, 0.01666667])

In [75]:
mean_score

0.1

In [76]:
# other metrics + graphs




### Hyper_optimized model

#### *leave-one-task-out* cross validation

##### Training

In [77]:
hm_linear_svm_to = hyper_model(model_name='linear_svm',
                            validation_approach = 'leave-one-task-out',
                            X=X,
                            y=y)

In [78]:
hm_linear_svm_to.fit()

Initial Model with Stock Params Score : 0.49375
Fitting 4 folds for each of 5 candidates, totalling 20 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:  2.0min finished


Random HP Search Best Model Score : 0.5166666666666666
Improvement of 4.641350210970447% relative to the initial model


In [79]:
hm_linear_svm_to.all_models_perf

,Model Iteration,Score


##### Test

In [80]:
# leave-one-task-out test
cv = KFold(n_splits=4,shuffle=False)

scores = cross_val_score(estimator=hm_linear_svm_to.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)



In [81]:
scores

array([0.48333333, 0.45833333, 0.49166667, 0.55      ])

In [82]:
mean_score

0.49583333333333335

In [83]:
# leave-one-task-out-shuffed test
cv = KFold(n_splits=4,shuffle=True,random_state=42)

scores = cross_val_score(estimator=hm_linear_svm_to.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [84]:
scores

array([0.125     , 0.11666667, 0.1       , 0.08333333])

In [85]:
mean_score

0.10625

In [86]:
# leave-one-run-out test
cv = KFold(n_splits=8,shuffle=False)

scores = cross_val_score(estimator=hm_linear_svm_to.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [87]:
scores

array([0.5       , 0.58333333, 0.41666667, 0.46666667, 0.53333333,
       0.48333333, 0.5       , 0.51666667])

In [88]:
mean_score

0.5

In [89]:
# leave-one-run-out-shuffed test
cv = KFold(n_splits=8,shuffle=True,random_state=42)



scores = cross_val_score(estimator=hm_linear_svm_to.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [90]:
scores

array([0.08333333, 0.13333333, 0.03333333, 0.08333333, 0.28333333,
       0.11666667, 0.06666667, 0.01666667])

In [91]:
mean_score

0.10208333333333333

In [92]:
# other metrics + graphs




#### *leave-one-task-out-shuffled* cross validation

##### Training

In [93]:
hm_linear_svm_tos = hyper_model(model_name='linear_svm',
                            validation_approach = 'leave-one-task-out-shuffled',
                            X=X,
                            y=y)

In [94]:
hm_linear_svm_tos.fit()

Initial Model with Stock Params Score : 0.10625
Fitting 4 folds for each of 5 candidates, totalling 20 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:   54.1s finished


Random HP Search Best Model Score : 0.10625
Improvement of 0.0% relative to the initial model


In [95]:
hm_linear_svm_tos.all_models_perf

,Model Iteration,Score


##### Test

In [96]:
# leave-one-task-out test
cv = KFold(n_splits=4,shuffle=False)

scores = cross_val_score(estimator=hm_linear_svm_tos.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)



In [97]:
scores

array([0.475     , 0.48333333, 0.525     , 0.45833333])

In [98]:
mean_score

0.48541666666666666

In [99]:
# leave-one-task-out-shuffed test
cv = KFold(n_splits=4,shuffle=True,random_state=42)

scores = cross_val_score(estimator=hm_linear_svm_tos.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [100]:
scores

array([0.125     , 0.11666667, 0.1       , 0.08333333])

In [101]:
mean_score

0.10625

In [102]:
# leave-one-run-out test
cv = KFold(n_splits=8,shuffle=False)

scores = cross_val_score(estimator=hm_linear_svm_tos.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [103]:
scores

array([0.56666667, 0.51666667, 0.4       , 0.5       , 0.58333333,
       0.56666667, 0.51666667, 0.55      ])

In [104]:
mean_score

0.525

In [105]:
# leave-one-run-out-shuffed test
cv = KFold(n_splits=8,shuffle=True,random_state=42)



scores = cross_val_score(estimator=hm_linear_svm_tos.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [106]:
scores

array([0.06666667, 0.11666667, 0.03333333, 0.08333333, 0.25      ,
       0.11666667, 0.06666667, 0.01666667])

In [107]:
mean_score

0.09375

In [108]:
# other metrics + graphs




#### *leave-one-run-out* cross validation

##### Training

In [109]:
hm_linear_svm_ro = hyper_model(model_name='linear_svm',
                            validation_approach = 'leave-one-run-out',
                            X=X,
                            y=y)

In [110]:
hm_linear_svm_ro.fit()

Initial Model with Stock Params Score : 0.525
Fitting 8 folds for each of 5 candidates, totalling 40 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:  4.7min finished


Random HP Search Best Model Score : 0.5145833333333334
Improvement of -1.984126984126977% relative to the initial model


In [111]:
hm_linear_svm_ro.all_models_perf

,Model Iteration,Score


##### Test

In [112]:
# leave-one-task-out test
cv = KFold(n_splits=4,shuffle=False)

scores = cross_val_score(estimator=hm_linear_svm_ro.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)



In [113]:
scores

array([0.51666667, 0.46666667, 0.5       , 0.48333333])

In [114]:
mean_score

0.4916666666666667

In [115]:
# leave-one-task-out-shuffed test
cv = KFold(n_splits=4,shuffle=True,random_state=42)

scores = cross_val_score(estimator=hm_linear_svm_ro.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [116]:
scores

array([0.125     , 0.11666667, 0.1       , 0.08333333])

In [117]:
mean_score

0.10625

In [118]:
# leave-one-run-out test
cv = KFold(n_splits=8,shuffle=False)

scores = cross_val_score(estimator=hm_linear_svm_ro.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [119]:
scores

array([0.5       , 0.53333333, 0.53333333, 0.45      , 0.53333333,
       0.5       , 0.45      , 0.48333333])

In [120]:
mean_score

0.4979166666666667

In [121]:
# leave-one-run-out-shuffed test
cv = KFold(n_splits=8,shuffle=True,random_state=42)



scores = cross_val_score(estimator=hm_linear_svm_ro.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [122]:
scores

array([0.08333333, 0.13333333, 0.03333333, 0.08333333, 0.26666667,
       0.11666667, 0.06666667, 0.01666667])

In [123]:
mean_score

0.1

In [124]:
# other metrics + graphs




#### *leave-one-run-out-shuffled* cross validation

##### Training

In [125]:
hm_linear_svm_ros = hyper_model(model_name='linear_svm',
                            validation_approach = 'leave-one-run-out-shuffled',
                            X=X,
                            y=y)

In [126]:
hm_linear_svm_ros.fit()

Initial Model with Stock Params Score : 0.1
Fitting 8 folds for each of 5 candidates, totalling 40 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:  4.4min finished


Random HP Search Best Model Score : 0.10208333333333333
Improvement of 2.083333333333326% relative to the initial model


In [127]:
hm_linear_svm_ros.all_models_perf

,Model Iteration,Score


##### Test

In [128]:
# leave-one-task-out test
cv = KFold(n_splits=4,shuffle=False)

scores = cross_val_score(estimator=hm_linear_svm_ros.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)



In [129]:
scores

array([0.46666667, 0.45      , 0.50833333, 0.49166667])

In [130]:
mean_score

0.4791666666666667

In [131]:
# leave-one-task-out-shuffed test
cv = KFold(n_splits=4,shuffle=True,random_state=42)

scores = cross_val_score(estimator=hm_linear_svm_ros.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [132]:
scores

array([0.125     , 0.11666667, 0.1       , 0.08333333])

In [133]:
mean_score

0.10625

In [134]:
# leave-one-run-out test
cv = KFold(n_splits=8,shuffle=False)

scores = cross_val_score(estimator=hm_linear_svm_ros.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [135]:
scores

array([0.5       , 0.5       , 0.46666667, 0.46666667, 0.45      ,
       0.51666667, 0.55      , 0.48333333])

In [136]:
mean_score

0.4916666666666667

In [137]:
# leave-one-run-out-shuffed test
cv = KFold(n_splits=8,shuffle=True,random_state=42)



scores = cross_val_score(estimator=hm_linear_svm_ros.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [138]:
scores

array([0.08333333, 0.13333333, 0.03333333, 0.08333333, 0.26666667,
       0.11666667, 0.06666667, 0.01666667])

In [139]:
mean_score

0.1

In [140]:
# other metrics + graphs




## Non Linear Support Vector Machine

### Stock Model

#### *leave-one-task-out* cross validation

##### Training

In [141]:
hm_non_linear_svm_to = hyper_model(model_name='non_linear_svm',
                            validation_approach = 'leave-one-task-out',
                            X=X,
                            y=y)

In [142]:
hm_non_linear_svm_to.fit()

Initial Model with Stock Params Score : 0.48750000000000004
Fitting 4 folds for each of 75 candidates, totalling 300 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:  9.3min finished


Random HP Search Best Model Score : 0.5291666666666667
Improvement of 8.547008547008538% relative to the initial model


In [143]:
hm_non_linear_svm_to.all_models_perf

,Model Iteration,Score


##### Test

In [144]:
# leave-one-task-out test
cv = KFold(n_splits=4,shuffle=False)

scores = cross_val_score(estimator=hm_non_linear_svm_to.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)



In [145]:
scores

array([0.58333333, 0.4       , 0.5       , 0.46666667])

In [146]:
mean_score

0.48750000000000004

In [147]:
# leave-one-task-out-shuffed test
cv = KFold(n_splits=4,shuffle=True,random_state=42)

scores = cross_val_score(estimator=hm_non_linear_svm_to.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [148]:
scores

array([0.23333333, 0.175     , 0.15833333, 0.2       ])

In [149]:
mean_score

0.19166666666666665

In [150]:
# leave-one-run-out test
cv = KFold(n_splits=8,shuffle=False)

scores = cross_val_score(estimator=hm_non_linear_svm_to.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [151]:
scores

array([0.6       , 0.5       , 0.35      , 0.41666667, 0.53333333,
       0.53333333, 0.53333333, 0.56666667])

In [152]:
mean_score

0.5041666666666667

In [153]:
# leave-one-run-out-shuffed test
cv = KFold(n_splits=8,shuffle=True,random_state=42)



scores = cross_val_score(estimator=hm_non_linear_svm_to.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [154]:
scores

array([0.15      , 0.21666667, 0.16666667, 0.11666667, 0.2       ,
       0.11666667, 0.23333333, 0.11666667])

In [155]:
mean_score

0.16458333333333333

In [156]:
# other metrics + graphs




#### *leave-one-task-out-shuffled* cross validation

##### Training

In [157]:
hm_non_linear_svm_tos = hyper_model(model_name='non_linear_svm',
                            validation_approach = 'leave-one-task-out-shuffled',
                            X=X,
                            y=y)

In [158]:
hm_non_linear_svm_tos.fit()

Initial Model with Stock Params Score : 0.19166666666666665
Fitting 4 folds for each of 75 candidates, totalling 300 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:  9.5min finished


Random HP Search Best Model Score : 0.7916666666666667
Improvement of 313.0434782608696% relative to the initial model


In [159]:
hm_non_linear_svm_tos.all_models_perf

,Model Iteration,Score


##### Test

In [160]:
# leave-one-task-out test
cv = KFold(n_splits=4,shuffle=False)

scores = cross_val_score(estimator=hm_non_linear_svm_tos.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)



In [161]:
scores

array([0.58333333, 0.4       , 0.5       , 0.46666667])

In [162]:
mean_score

0.48750000000000004

In [163]:
# leave-one-task-out-shuffed test
cv = KFold(n_splits=4,shuffle=True,random_state=42)

scores = cross_val_score(estimator=hm_non_linear_svm_tos.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [164]:
scores

array([0.23333333, 0.175     , 0.15833333, 0.2       ])

In [165]:
mean_score

0.19166666666666665

In [166]:
# leave-one-run-out test
cv = KFold(n_splits=8,shuffle=False)

scores = cross_val_score(estimator=hm_non_linear_svm_tos.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [167]:
scores

array([0.6       , 0.5       , 0.35      , 0.41666667, 0.53333333,
       0.53333333, 0.53333333, 0.56666667])

In [168]:
mean_score

0.5041666666666667

In [169]:
# leave-one-run-out-shuffed test
cv = KFold(n_splits=8,shuffle=True,random_state=42)



scores = cross_val_score(estimator=hm_non_linear_svm_tos.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [170]:
scores

array([0.15      , 0.21666667, 0.16666667, 0.11666667, 0.2       ,
       0.11666667, 0.23333333, 0.11666667])

In [171]:
mean_score

0.16458333333333333

In [172]:
# other metrics + graphs




#### *leave-one-run-out* cross validation

##### Training

In [173]:
hm_non_linear_svm_ro = hyper_model(model_name='non_linear_svm',
                            validation_approach = 'leave-one-run-out',
                            X=X,
                            y=y)

In [174]:
hm_non_linear_svm_ro.fit()

Initial Model with Stock Params Score : 0.5041666666666667
Fitting 8 folds for each of 75 candidates, totalling 600 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 600 out of 600 | elapsed: 28.5min finished


Random HP Search Best Model Score : 0.55
Improvement of 9.090909090909102% relative to the initial model


In [175]:
hm_non_linear_svm_ro.all_models_perf

,Model Iteration,Score


##### Test

In [176]:
# leave-one-task-out test
cv = KFold(n_splits=4,shuffle=False)

scores = cross_val_score(estimator=hm_non_linear_svm_ro.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)



In [177]:
scores

array([0.58333333, 0.4       , 0.5       , 0.46666667])

In [178]:
mean_score

0.48750000000000004

In [179]:
# leave-one-task-out-shuffed test
cv = KFold(n_splits=4,shuffle=True,random_state=42)

scores = cross_val_score(estimator=hm_non_linear_svm_ro.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [180]:
scores

array([0.23333333, 0.175     , 0.15833333, 0.2       ])

In [181]:
mean_score

0.19166666666666665

In [182]:
# leave-one-run-out test
cv = KFold(n_splits=8,shuffle=False)

scores = cross_val_score(estimator=hm_non_linear_svm_ro.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [183]:
scores

array([0.6       , 0.5       , 0.35      , 0.41666667, 0.53333333,
       0.53333333, 0.53333333, 0.56666667])

In [184]:
mean_score

0.5041666666666667

In [185]:
# leave-one-run-out-shuffed test
cv = KFold(n_splits=8,shuffle=True,random_state=42)



scores = cross_val_score(estimator=hm_non_linear_svm_ro.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [186]:
scores

array([0.15      , 0.21666667, 0.16666667, 0.11666667, 0.2       ,
       0.11666667, 0.23333333, 0.11666667])

In [187]:
mean_score

0.16458333333333333

In [188]:
# other metrics + graphs




#### *leave-one-run-out-shuffled* cross validation

##### Training

In [189]:
hm_non_linear_svm_ros = hyper_model(model_name='non_linear_svm',
                            validation_approach = 'leave-one-run-out-shuffled',
                            X=X,
                            y=y)

In [190]:
hm_non_linear_svm_ros.fit()

Initial Model with Stock Params Score : 0.16458333333333333
Fitting 8 folds for each of 75 candidates, totalling 600 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 600 out of 600 | elapsed: 31.6min finished


Random HP Search Best Model Score : 0.8708333333333333
Improvement of 429.113924050633% relative to the initial model


In [191]:
hm_non_linear_svm_ros.all_models_perf

,Model Iteration,Score


##### Test

In [192]:
# leave-one-task-out test
cv = KFold(n_splits=4,shuffle=False)

scores = cross_val_score(estimator=hm_non_linear_svm_ros.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)



In [193]:
scores

array([0.58333333, 0.4       , 0.5       , 0.46666667])

In [194]:
mean_score

0.48750000000000004

In [195]:
# leave-one-task-out-shuffed test
cv = KFold(n_splits=4,shuffle=True,random_state=42)

scores = cross_val_score(estimator=hm_non_linear_svm_ros.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [196]:
scores

array([0.23333333, 0.175     , 0.15833333, 0.2       ])

In [197]:
mean_score

0.19166666666666665

In [198]:
# leave-one-run-out test
cv = KFold(n_splits=8,shuffle=False)

scores = cross_val_score(estimator=hm_non_linear_svm_ros.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [199]:
scores

array([0.6       , 0.5       , 0.35      , 0.41666667, 0.53333333,
       0.53333333, 0.53333333, 0.56666667])

In [200]:
mean_score

0.5041666666666667

In [201]:
# leave-one-run-out-shuffed test
cv = KFold(n_splits=8,shuffle=True,random_state=42)



scores = cross_val_score(estimator=hm_non_linear_svm_ros.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [202]:
scores

array([0.15      , 0.21666667, 0.16666667, 0.11666667, 0.2       ,
       0.11666667, 0.23333333, 0.11666667])

In [203]:
mean_score

0.16458333333333333

In [204]:
# other metrics + graphs




### Hyper_optimized model

### Stock Model

#### *leave-one-task-out* cross validation

##### Training

In [205]:
hm_non_linear_svm_to = hyper_model(model_name='non_linear_svm',
                            validation_approach = 'leave-one-task-out',
                            X=X,
                            y=y)

In [206]:
hm_non_linear_svm_to.fit()

Initial Model with Stock Params Score : 0.48750000000000004
Fitting 4 folds for each of 75 candidates, totalling 300 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:  9.4min finished


Random HP Search Best Model Score : 0.5291666666666667
Improvement of 8.547008547008538% relative to the initial model


In [207]:
hm_non_linear_svm_to.all_models_perf

,Model Iteration,Score


##### Test

In [208]:
# leave-one-task-out test
cv = KFold(n_splits=4,shuffle=False)

scores = cross_val_score(estimator=hm_non_linear_svm_to.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)



In [209]:
scores

array([0.55833333, 0.45      , 0.575     , 0.53333333])

In [210]:
mean_score

0.5291666666666667

In [211]:
# leave-one-task-out-shuffed test
cv = KFold(n_splits=4,shuffle=True,random_state=42)

scores = cross_val_score(estimator=hm_non_linear_svm_to.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [212]:
scores

array([0.45      , 0.39166667, 0.45833333, 0.45      ])

In [213]:
mean_score

0.4375

In [214]:
# leave-one-run-out test
cv = KFold(n_splits=8,shuffle=False)

scores = cross_val_score(estimator=hm_non_linear_svm_to.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [215]:
scores

array([0.53333333, 0.5       , 0.5       , 0.4       , 0.56666667,
       0.55      , 0.48333333, 0.48333333])

In [216]:
mean_score

0.5020833333333333

In [217]:
# leave-one-run-out-shuffed test
cv = KFold(n_splits=8,shuffle=True,random_state=42)



scores = cross_val_score(estimator=hm_non_linear_svm_to.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [218]:
scores

array([0.35      , 0.45      , 0.56666667, 0.38333333, 0.41666667,
       0.51666667, 0.45      , 0.36666667])

In [219]:
mean_score

0.4375

In [220]:
# other metrics + graphs




#### *leave-one-task-out-shuffled* cross validation

##### Training

In [221]:
hm_non_linear_svm_tos = hyper_model(model_name='non_linear_svm',
                            validation_approach = 'leave-one-task-out-shuffled',
                            X=X,
                            y=y)

In [222]:
hm_non_linear_svm_tos.fit()

Initial Model with Stock Params Score : 0.19166666666666665
Fitting 4 folds for each of 75 candidates, totalling 300 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed: 10.6min finished


Random HP Search Best Model Score : 0.7916666666666667
Improvement of 313.0434782608696% relative to the initial model


In [223]:
hm_non_linear_svm_tos.all_models_perf

,Model Iteration,Score


##### Test

In [224]:
# leave-one-task-out test
cv = KFold(n_splits=4,shuffle=False)

scores = cross_val_score(estimator=hm_non_linear_svm_tos.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)



In [225]:
scores

array([0.425     , 0.45833333, 0.50833333, 0.55      ])

In [226]:
mean_score

0.48541666666666666

In [227]:
# leave-one-task-out-shuffed test
cv = KFold(n_splits=4,shuffle=True,random_state=42)

scores = cross_val_score(estimator=hm_non_linear_svm_tos.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [228]:
scores

array([0.83333333, 0.74166667, 0.78333333, 0.80833333])

In [229]:
mean_score

0.7916666666666667

In [230]:
# leave-one-run-out test
cv = KFold(n_splits=8,shuffle=False)

scores = cross_val_score(estimator=hm_non_linear_svm_tos.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [231]:
scores

array([0.45      , 0.43333333, 0.46666667, 0.56666667, 0.48333333,
       0.51666667, 0.51666667, 0.53333333])

In [232]:
mean_score

0.4958333333333333

In [233]:
# leave-one-run-out-shuffed test
cv = KFold(n_splits=8,shuffle=True,random_state=42)



scores = cross_val_score(estimator=hm_non_linear_svm_tos.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [234]:
scores

array([0.75      , 0.83333333, 0.83333333, 0.75      , 0.76666667,
       0.78333333, 0.75      , 0.71666667])

In [235]:
mean_score

0.7729166666666667

In [236]:
# other metrics + graphs




#### *leave-one-run-out* cross validation

##### Training

In [237]:
hm_non_linear_svm_ro = hyper_model(model_name='non_linear_svm',
                            validation_approach = 'leave-one-run-out',
                            X=X,
                            y=y)

In [238]:
hm_non_linear_svm_ro.fit()

Initial Model with Stock Params Score : 0.5041666666666667
Fitting 8 folds for each of 75 candidates, totalling 600 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 600 out of 600 | elapsed: 30.6min finished


Random HP Search Best Model Score : 0.55
Improvement of 9.090909090909102% relative to the initial model


In [239]:
hm_non_linear_svm_ro.all_models_perf

,Model Iteration,Score


##### Test

In [240]:
# leave-one-task-out test
cv = KFold(n_splits=4,shuffle=False)

scores = cross_val_score(estimator=hm_non_linear_svm_ro.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)



In [241]:
scores

array([0.55833333, 0.36666667, 0.50833333, 0.23333333])

In [242]:
mean_score

0.4166666666666667

In [243]:
# leave-one-task-out-shuffed test
cv = KFold(n_splits=4,shuffle=True,random_state=42)

scores = cross_val_score(estimator=hm_non_linear_svm_ro.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [244]:
scores

array([0.51666667, 0.53333333, 0.61666667, 0.53333333])

In [245]:
mean_score

0.55

In [246]:
# leave-one-run-out test
cv = KFold(n_splits=8,shuffle=False)

scores = cross_val_score(estimator=hm_non_linear_svm_ro.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [247]:
scores

array([0.63333333, 0.48333333, 0.35      , 0.38333333, 0.5       ,
       0.51666667, 0.8       , 0.73333333])

In [248]:
mean_score

0.55

In [249]:
# leave-one-run-out-shuffed test
cv = KFold(n_splits=8,shuffle=True,random_state=42)



scores = cross_val_score(estimator=hm_non_linear_svm_ro.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [250]:
scores

array([0.55      , 0.48333333, 0.5       , 0.56666667, 0.63333333,
       0.6       , 0.53333333, 0.53333333])

In [251]:
mean_score

0.55

In [252]:
# other metrics + graphs




#### *leave-one-run-out-shuffled* cross validation

##### Training

In [253]:
hm_non_linear_svm_ros = hyper_model(model_name='non_linear_svm',
                            validation_approach = 'leave-one-run-out-shuffled',
                            X=X,
                            y=y)

In [254]:
hm_non_linear_svm_ros.fit()

Initial Model with Stock Params Score : 0.16458333333333333
Fitting 8 folds for each of 75 candidates, totalling 600 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 600 out of 600 | elapsed: 32.1min finished


Random HP Search Best Model Score : 0.8708333333333333
Improvement of 429.113924050633% relative to the initial model


In [255]:
hm_non_linear_svm_ros.all_models_perf

,Model Iteration,Score


##### Test

In [256]:
# leave-one-task-out test
cv = KFold(n_splits=4,shuffle=False)

scores = cross_val_score(estimator=hm_non_linear_svm_ros.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)



In [257]:
scores

array([0.46666667, 0.51666667, 0.53333333, 0.45833333])

In [258]:
mean_score

0.49374999999999997

In [259]:
# leave-one-task-out-shuffed test
cv = KFold(n_splits=4,shuffle=True,random_state=42)

scores = cross_val_score(estimator=hm_non_linear_svm_ros.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [260]:
scores

array([0.81666667, 0.80833333, 0.89166667, 0.13333333])

In [261]:
mean_score

0.6625

In [262]:
# leave-one-run-out test
cv = KFold(n_splits=8,shuffle=False)

scores = cross_val_score(estimator=hm_non_linear_svm_ros.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [263]:
scores

array([0.48333333, 0.48333333, 0.48333333, 0.48333333, 0.5       ,
       0.55      , 0.46666667, 0.53333333])

In [264]:
mean_score

0.4979166666666667

In [265]:
# leave-one-run-out-shuffed test
cv = KFold(n_splits=8,shuffle=True,random_state=42)



scores = cross_val_score(estimator=hm_non_linear_svm_ros.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [266]:
scores

array([0.91666667, 0.86666667, 0.85      , 0.85      , 0.91666667,
       0.86666667, 0.83333333, 0.86666667])

In [267]:
mean_score

0.8708333333333333

In [268]:
# other metrics + graphs




## Random Forest Classifier

### Stock Model

#### *leave-one-task-out* cross validation

##### Training

In [275]:
hm_rf_to = hyper_model(model_name='random_forest_classifier',
                            validation_approach = 'leave-one-task-out',
                            X=X,
                            y=y)

In [276]:
hm_rf_to.fit()

Initial Model with Stock Params Score : 0.4645833333333334
Fitting 4 folds for each of 2 candidates, totalling 8 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  1.4min finished


Random HP Search Best Model Score : 0.44166666666666665
Improvement of -4.932735426008986% relative to the initial model


In [277]:
hm_rf_to.all_models_perf

,Model Iteration,Score


##### Test

In [278]:
# leave-one-task-out test
cv = KFold(n_splits=4,shuffle=False)

scores = cross_val_score(estimator=hm_rf_to.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)



In [279]:
scores

array([0.56666667, 0.40833333, 0.48333333, 0.4       ])

In [280]:
mean_score

0.46458333333333335

In [281]:
# leave-one-task-out-shuffed test
cv = KFold(n_splits=4,shuffle=True,random_state=42)

scores = cross_val_score(estimator=hm_rf_to.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [282]:
scores

array([0.425     , 0.38333333, 0.325     , 0.425     ])

In [283]:
mean_score

0.38958333333333334

In [284]:
# leave-one-run-out test
cv = KFold(n_splits=8,shuffle=False)

scores = cross_val_score(estimator=hm_rf_to.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [285]:
scores

array([0.55      , 0.55      , 0.35      , 0.38333333, 0.53333333,
       0.5       , 0.51666667, 0.51666667])

In [286]:
mean_score

0.4875

In [287]:
# leave-one-run-out-shuffed test
cv = KFold(n_splits=8,shuffle=True,random_state=42)



scores = cross_val_score(estimator=hm_rf_to.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [288]:
scores

array([0.41666667, 0.43333333, 0.45      , 0.41666667, 0.28333333,
       0.46666667, 0.43333333, 0.31666667])

In [289]:
mean_score

0.40208333333333335

In [290]:
# other metrics + graphs




#### *leave-one-task-out-shuffled* cross validation

##### Training

In [291]:
hm_rf_tos = hyper_model(model_name='random_forest_classifier',
                            validation_approach = 'leave-one-task-out-shuffled',
                            X=X,
                            y=y)

In [292]:
hm_rf_tos.fit()

Initial Model with Stock Params Score : 0.37916666666666665
Fitting 4 folds for each of 2 candidates, totalling 8 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  1.4min finished


Random HP Search Best Model Score : 0.4166666666666667
Improvement of 9.8901098901099% relative to the initial model


In [293]:
hm_rf_tos.all_models_perf

,Model Iteration,Score


##### Test

In [294]:
# leave-one-task-out test
cv = KFold(n_splits=4,shuffle=False)

scores = cross_val_score(estimator=hm_rf_tos.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)



In [295]:
scores

array([0.53333333, 0.4       , 0.54166667, 0.45833333])

In [296]:
mean_score

0.48333333333333334

In [297]:
# leave-one-task-out-shuffed test
cv = KFold(n_splits=4,shuffle=True,random_state=42)

scores = cross_val_score(estimator=hm_rf_tos.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [298]:
scores

array([0.4       , 0.40833333, 0.34166667, 0.34166667])

In [299]:
mean_score

0.3729166666666667

In [300]:
# leave-one-run-out test
cv = KFold(n_splits=8,shuffle=False)

scores = cross_val_score(estimator=hm_rf_tos.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [301]:
scores

array([0.63333333, 0.51666667, 0.35      , 0.38333333, 0.55      ,
       0.48333333, 0.56666667, 0.58333333])

In [302]:
mean_score

0.5083333333333333

In [303]:
# leave-one-run-out-shuffed test
cv = KFold(n_splits=8,shuffle=True,random_state=42)



scores = cross_val_score(estimator=hm_rf_tos.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [304]:
scores

array([0.3       , 0.41666667, 0.38333333, 0.43333333, 0.48333333,
       0.41666667, 0.36666667, 0.33333333])

In [305]:
mean_score

0.39166666666666666

In [306]:
# other metrics + graphs




#### *leave-one-run-out* cross validation

##### Training

In [307]:
hm_rf_ro = hyper_model(model_name='random_forest_classifier',
                            validation_approach = 'leave-one-run-out',
                            X=X,
                            y=y)

In [308]:
hm_rf_ro.fit()

Initial Model with Stock Params Score : 0.4833333333333334
Fitting 8 folds for each of 2 candidates, totalling 16 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:  3.5min finished


Random HP Search Best Model Score : 0.5145833333333334
Improvement of 6.46551724137931% relative to the initial model


In [309]:
hm_rf_ro.all_models_perf

,Model Iteration,Score


##### Test

In [310]:
# leave-one-task-out test
cv = KFold(n_splits=4,shuffle=False)

scores = cross_val_score(estimator=hm_rf_ro.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)



In [311]:
scores

array([0.475     , 0.41666667, 0.49166667, 0.39166667])

In [312]:
mean_score

0.44375

In [313]:
# leave-one-task-out-shuffed test
cv = KFold(n_splits=4,shuffle=True,random_state=42)

scores = cross_val_score(estimator=hm_rf_ro.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [314]:
scores

array([0.38333333, 0.35833333, 0.45      , 0.39166667])

In [315]:
mean_score

0.3958333333333333

In [316]:
# leave-one-run-out test
cv = KFold(n_splits=8,shuffle=False)

scores = cross_val_score(estimator=hm_rf_ro.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [317]:
scores

array([0.53333333, 0.48333333, 0.31666667, 0.43333333, 0.53333333,
       0.51666667, 0.46666667, 0.55      ])

In [318]:
mean_score

0.47916666666666663

In [319]:
# leave-one-run-out-shuffed test
cv = KFold(n_splits=8,shuffle=True,random_state=42)



scores = cross_val_score(estimator=hm_rf_ro.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [320]:
scores

array([0.43333333, 0.41666667, 0.33333333, 0.38333333, 0.33333333,
       0.4       , 0.33333333, 0.25      ])

In [321]:
mean_score

0.3604166666666667

In [322]:
# other metrics + graphs




#### *leave-one-run-out-shuffled* cross validation

##### Training

In [323]:
hm_rf_ros = hyper_model(model_name='random_forest_classifier',
                            validation_approach = 'leave-one-run-out-shuffled',
                            X=X,
                            y=y)

In [324]:
hm_rf_ros.fit()

Initial Model with Stock Params Score : 0.39375
Fitting 8 folds for each of 2 candidates, totalling 16 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:  3.8min finished


Random HP Search Best Model Score : 0.4291666666666667
Improvement of 8.994708994709006% relative to the initial model


In [325]:
hm_rf_ros.all_models_perf

,Model Iteration,Score


##### Test

In [326]:
# leave-one-task-out test
cv = KFold(n_splits=4,shuffle=False)

scores = cross_val_score(estimator=hm_rf_ros.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)



In [327]:
scores

array([0.51666667, 0.35      , 0.55833333, 0.34166667])

In [328]:
mean_score

0.44166666666666665

In [329]:
# leave-one-task-out-shuffed test
cv = KFold(n_splits=4,shuffle=True,random_state=42)

scores = cross_val_score(estimator=hm_rf_ros.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [330]:
scores

array([0.35833333, 0.4       , 0.325     , 0.44166667])

In [331]:
mean_score

0.38125

In [332]:
# leave-one-run-out test
cv = KFold(n_splits=8,shuffle=False)

scores = cross_val_score(estimator=hm_rf_ros.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [333]:
scores

array([0.58333333, 0.46666667, 0.38333333, 0.41666667, 0.48333333,
       0.53333333, 0.55      , 0.5       ])

In [334]:
mean_score

0.4895833333333333

In [335]:
# leave-one-run-out-shuffed test
cv = KFold(n_splits=8,shuffle=True,random_state=42)



scores = cross_val_score(estimator=hm_rf_ros.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [336]:
scores

array([0.31666667, 0.33333333, 0.3       , 0.46666667, 0.41666667,
       0.31666667, 0.28333333, 0.41666667])

In [337]:
mean_score

0.35624999999999996

In [338]:
# other metrics + graphs




### Hyper_optimized Model

### Stock Model

#### *leave-one-task-out* cross validation

##### Training

In [339]:
hm_rf_to = hyper_model(model_name='random_forest_classifier',
                            validation_approach = 'leave-one-task-out',
                            X=X,
                            y=y)

In [340]:
hm_rf_to.fit()

Initial Model with Stock Params Score : 0.45624999999999993
Fitting 4 folds for each of 2 candidates, totalling 8 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  1.9min finished


Random HP Search Best Model Score : 0.44375
Improvement of -2.7397260273972512% relative to the initial model


In [341]:
hm_rf_to.all_models_perf

,Model Iteration,Score


##### Test

In [342]:
# leave-one-task-out test
cv = KFold(n_splits=4,shuffle=False)

scores = cross_val_score(estimator=hm_rf_to.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)



In [343]:
scores

array([0.56666667, 0.36666667, 0.50833333, 0.29166667])

In [344]:
mean_score

0.43333333333333335

In [345]:
# leave-one-task-out-shuffed test
cv = KFold(n_splits=4,shuffle=True,random_state=42)

scores = cross_val_score(estimator=hm_rf_to.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [346]:
scores

array([0.44166667, 0.49166667, 0.28333333, 0.4       ])

In [347]:
mean_score

0.4041666666666667

In [348]:
# leave-one-run-out test
cv = KFold(n_splits=8,shuffle=False)

scores = cross_val_score(estimator=hm_rf_to.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [349]:
scores

array([0.65      , 0.48333333, 0.35      , 0.38333333, 0.5       ,
       0.51666667, 0.51666667, 0.68333333])

In [350]:
mean_score

0.5104166666666667

In [351]:
# leave-one-run-out-shuffed test
cv = KFold(n_splits=8,shuffle=True,random_state=42)



scores = cross_val_score(estimator=hm_rf_to.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [352]:
scores

array([0.41666667, 0.41666667, 0.35      , 0.43333333, 0.45      ,
       0.33333333, 0.43333333, 0.4       ])

In [353]:
mean_score

0.4041666666666667

In [354]:
# other metrics + graphs




#### *leave-one-task-out-shuffled* cross validation

##### Training

In [355]:
hm_rf_tos = hyper_model(model_name='random_forest_classifier',
                            validation_approach = 'leave-one-task-out-shuffled',
                            X=X,
                            y=y)

In [356]:
hm_rf_tos.fit()

Initial Model with Stock Params Score : 0.375
Fitting 4 folds for each of 2 candidates, totalling 8 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  2.0min finished


Random HP Search Best Model Score : 0.39166666666666666
Improvement of 4.444444444444443% relative to the initial model


In [357]:
hm_rf_tos.all_models_perf

,Model Iteration,Score


##### Test

In [358]:
# leave-one-task-out test
cv = KFold(n_splits=4,shuffle=False)

scores = cross_val_score(estimator=hm_rf_tos.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)



In [359]:
scores

array([0.58333333, 0.36666667, 0.5       , 0.31666667])

In [360]:
mean_score

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.

0.44166666666666665

In [361]:
# leave-one-task-out-shuffed test
cv = KFold(n_splits=4,shuffle=True,random_state=42)

scores = cross_val_score(estimator=hm_rf_tos.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [362]:
scores

array([0.43333333, 0.49166667, 0.225     , 0.43333333])

In [363]:
mean_score

0.39583333333333337

In [364]:
# leave-one-run-out test
cv = KFold(n_splits=8,shuffle=False)

scores = cross_val_score(estimator=hm_rf_tos.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [365]:
scores

array([0.63333333, 0.5       , 0.35      , 0.38333333, 0.5       ,
       0.51666667, 0.6       , 0.66666667])

In [366]:
mean_score

0.51875

In [367]:
# leave-one-run-out-shuffed test
cv = KFold(n_splits=8,shuffle=True,random_state=42)



scores = cross_val_score(estimator=hm_rf_tos.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [368]:
scores

array([0.43333333, 0.41666667, 0.33333333, 0.48333333, 0.43333333,
       0.38333333, 0.48333333, 0.41666667])

In [369]:
mean_score

0.4229166666666667

In [370]:
# other metrics + graphs




#### *leave-one-run-out* cross validation

##### Training

In [371]:
hm_rf_ro = hyper_model(model_name='random_forest_classifier',
                            validation_approach = 'leave-one-run-out',
                            X=X,
                            y=y)

In [372]:
hm_rf_ro.fit()

Initial Model with Stock Params Score : 0.5020833333333333
Fitting 8 folds for each of 2 candidates, totalling 16 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:  5.5min finished


Random HP Search Best Model Score : 0.5270833333333333
Improvement of 4.9792531120331995% relative to the initial model


In [373]:
hm_rf_ro.all_models_perf

,Model Iteration,Score


##### Test

In [374]:
# leave-one-task-out test
cv = KFold(n_splits=4,shuffle=False)

scores = cross_val_score(estimator=hm_rf_ro.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)



In [375]:
scores

array([0.55833333, 0.36666667, 0.50833333, 0.375     ])

In [376]:
mean_score

0.45208333333333334

In [377]:
# leave-one-task-out-shuffed test
cv = KFold(n_splits=4,shuffle=True,random_state=42)

scores = cross_val_score(estimator=hm_rf_ro.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [378]:
scores

array([0.425     , 0.49166667, 0.275     , 0.38333333])

In [379]:
mean_score

0.39375

In [380]:
# leave-one-run-out test
cv = KFold(n_splits=8,shuffle=False)

scores = cross_val_score(estimator=hm_rf_ro.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [381]:
scores

array([0.63333333, 0.46666667, 0.35      , 0.36666667, 0.5       ,
       0.51666667, 0.6       , 0.63333333])

In [382]:
mean_score

0.5083333333333333

In [383]:
# leave-one-run-out-shuffed test
cv = KFold(n_splits=8,shuffle=True,random_state=42)



scores = cross_val_score(estimator=hm_rf_ro.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [384]:
scores

array([0.4       , 0.4       , 0.35      , 0.48333333, 0.41666667,
       0.45      , 0.45      , 0.4       ])

In [385]:
mean_score

0.41875

In [386]:
# other metrics + graphs




#### *leave-one-run-out-shuffled* cross validation

##### Training

In [387]:
hm_rf_ros = hyper_model(model_name='random_forest_classifier',
                            validation_approach = 'leave-one-run-out-shuffled',
                            X=X,
                            y=y)

In [388]:
hm_rf_ros.fit()

Initial Model with Stock Params Score : 0.3833333333333333
Fitting 8 folds for each of 2 candidates, totalling 16 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:  5.4min finished


Random HP Search Best Model Score : 0.42083333333333334
Improvement of 9.782608695652184% relative to the initial model


In [389]:
hm_rf_ros.all_models_perf

,Model Iteration,Score


##### Test

In [390]:
# leave-one-task-out test
cv = KFold(n_splits=4,shuffle=False)

scores = cross_val_score(estimator=hm_rf_ros.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)



In [391]:
scores

array([0.55      , 0.36666667, 0.50833333, 0.34166667])

In [392]:
mean_score

0.44166666666666665

In [393]:
# leave-one-task-out-shuffed test
cv = KFold(n_splits=4,shuffle=True,random_state=42)

scores = cross_val_score(estimator=hm_rf_ros.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [394]:
scores

array([0.44166667, 0.475     , 0.29166667, 0.38333333])

In [395]:
mean_score

0.39791666666666664

In [396]:
# leave-one-run-out test
cv = KFold(n_splits=8,shuffle=False)

scores = cross_val_score(estimator=hm_rf_ros.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [397]:
scores

array([0.61666667, 0.48333333, 0.35      , 0.38333333, 0.5       ,
       0.55      , 0.6       , 0.68333333])

In [398]:
mean_score

0.5208333333333333

In [399]:
# leave-one-run-out-shuffed test
cv = KFold(n_splits=8,shuffle=True,random_state=42)



scores = cross_val_score(estimator=hm_rf_ros.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


In [400]:
scores

array([0.36666667, 0.45      , 0.4       , 0.48333333, 0.43333333,
       0.3       , 0.35      , 0.43333333])

In [401]:
mean_score

0.40208333333333335

In [402]:
# other metrics + graphs




## XGBoost Classifier

### Stock Model

#### *leave-one-task-out* cross validation

##### Training

In [403]:
hm_xgb_to = hyper_model(model_name='xg_boost',
                            validation_approach = 'leave-one-task-out',
                            X=X,
                            y=y)

In [404]:
hm_xgb_to.fit()

[08:42:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[08:42:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[08:42:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[08:42:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[08:42:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[08:42:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[08:42:44] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[08:42:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[08:44:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[08:44:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[08:44:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[08:45:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[08:47:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[08:47:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[08:47:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[08:47:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

...

[10:21:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:21:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:21:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:21:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[10:23:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:23:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:24:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:24:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[10:26:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:26:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:26:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:26:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[Parallel(n_jobs=1)]: Done 1200 out of 1200 | elapsed: 104.5min finished


[10:27:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Random HP Search Best Model Score : 0.5229166666666667
Improvement of 9.606986899563331% relative to the initial model


In [405]:
hm_xgb_to.all_models_perf

,Model Iteration,Score


##### Test

In [406]:
# leave-one-task-out test
cv = KFold(n_splits=4,shuffle=False)

scores = cross_val_score(estimator=hm_xgb_to.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)



[10:27:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:27:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:27:17] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:27:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [407]:
scores

array([0.56666667, 0.39166667, 0.49166667, 0.45833333])

In [408]:
mean_score

0.4770833333333333

In [409]:
# leave-one-task-out-shuffed test
cv = KFold(n_splits=4,shuffle=True,random_state=42)

scores = cross_val_score(estimator=hm_xgb_to.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


[10:27:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:27:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:27:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:27:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [410]:
scores

array([0.29166667, 0.28333333, 0.29166667, 0.29166667])

In [411]:
mean_score

0.28958333333333336

In [412]:
# leave-one-run-out test
cv = KFold(n_splits=8,shuffle=False)

scores = cross_val_score(estimator=hm_xgb_to.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


[10:27:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:27:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:27:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:27:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [413]:
scores

array([0.46666667, 0.48333333, 0.33333333, 0.33333333, 0.53333333,
       0.56666667, 0.5       , 0.51666667])

In [414]:
mean_score

0.4666666666666667

In [415]:
# leave-one-run-out-shuffed test
cv = KFold(n_splits=8,shuffle=True,random_state=42)



scores = cross_val_score(estimator=hm_xgb_to.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


[10:28:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:28:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:28:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:28:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [416]:
scores

array([0.28333333, 0.23333333, 0.33333333, 0.31666667, 0.26666667,
       0.23333333, 0.28333333, 0.25      ])

In [417]:
mean_score

0.27499999999999997

In [418]:
# other metrics + graphs




#### *leave-one-task-out-shuffled* cross validation

##### Training

In [419]:
hm_xgb_tos = hyper_model(model_name='xg_boost',
                            validation_approach = 'leave-one-task-out-shuffled',
                            X=X,
                            y=y)

In [420]:
hm_xgb_tos.fit()

[10:29:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:29:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:29:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:29:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[10:29:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:29:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:29:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:29:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[10:31:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:31:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:31:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:31:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[10:33:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:33:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:34:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:34:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

...

[12:07:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:07:50] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:07:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:07:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[12:09:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:09:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:09:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:10:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[12:12:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:12:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:12:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:12:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[Parallel(n_jobs=1)]: Done 1200 out of 1200 | elapsed: 103.5min finished


[12:12:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Random HP Search Best Model Score : 0.3145833333333334
Improvement of 8.633093525179863% relative to the initial model


In [421]:
hm_xgb_tos.all_models_perf

,Model Iteration,Score


##### Test

In [422]:
# leave-one-task-out test
cv = KFold(n_splits=4,shuffle=False)

scores = cross_val_score(estimator=hm_xgb_tos.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)



[12:13:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:13:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:13:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:13:17] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [423]:
scores

array([0.56666667, 0.39166667, 0.49166667, 0.45833333])

In [424]:
mean_score

0.4770833333333333

In [425]:
# leave-one-task-out-shuffed test
cv = KFold(n_splits=4,shuffle=True,random_state=42)

scores = cross_val_score(estimator=hm_xgb_tos.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


[12:13:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:13:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:13:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:13:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [426]:
scores

array([0.29166667, 0.28333333, 0.29166667, 0.29166667])

In [427]:
mean_score

0.28958333333333336

In [428]:
# leave-one-run-out test
cv = KFold(n_splits=8,shuffle=False)

scores = cross_val_score(estimator=hm_xgb_tos.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


[12:13:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:13:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:13:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:13:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [429]:
scores

array([0.46666667, 0.48333333, 0.33333333, 0.33333333, 0.53333333,
       0.56666667, 0.5       , 0.51666667])

In [430]:
mean_score

0.4666666666666667

In [431]:
# leave-one-run-out-shuffed test
cv = KFold(n_splits=8,shuffle=True,random_state=42)



scores = cross_val_score(estimator=hm_xgb_tos.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


[12:14:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:14:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:14:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:14:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [432]:
scores

array([0.28333333, 0.23333333, 0.33333333, 0.31666667, 0.26666667,
       0.23333333, 0.28333333, 0.25      ])

In [433]:
mean_score

0.27499999999999997

In [434]:
# other metrics + graphs




#### *leave-one-run-out* cross validation

##### Training

In [438]:
hm_xgb_ro = hyper_model(model_name='xg_boost',
                            validation_approach = 'leave-one-run-out',
                            X=X,
                            y=y)

In [439]:
hm_xgb_ro.fit()

[15:38:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:38:17] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:38:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:38:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[15:39:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:39:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:39:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:40:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:  3.8min finished


[15:43:17] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Random HP Search Best Model Score : 0.49166666666666664
Improvement of 5.35714285714285% relative to the initial model


In [440]:
hm_xgb_ro.all_models_perf

,Model Iteration,Score


##### Test

In [441]:
# leave-one-task-out test
cv = KFold(n_splits=4,shuffle=False)

scores = cross_val_score(estimator=hm_xgb_ro.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)



[15:43:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:43:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:43:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:43:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [442]:
scores

array([0.56666667, 0.39166667, 0.49166667, 0.45833333])

In [443]:
mean_score

0.4770833333333333

In [444]:
# leave-one-task-out-shuffed test
cv = KFold(n_splits=4,shuffle=True,random_state=42)

scores = cross_val_score(estimator=hm_xgb_ro.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


[15:44:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:44:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:44:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:44:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [445]:
scores

array([0.29166667, 0.28333333, 0.29166667, 0.29166667])

In [446]:
mean_score

0.28958333333333336

In [447]:
# leave-one-run-out test
cv = KFold(n_splits=8,shuffle=False)

scores = cross_val_score(estimator=hm_xgb_ro.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


[15:44:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:44:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:44:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:45:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [448]:
scores

array([0.46666667, 0.48333333, 0.33333333, 0.33333333, 0.53333333,
       0.56666667, 0.5       , 0.51666667])

In [449]:
mean_score

0.4666666666666667

In [450]:
# leave-one-run-out-shuffed test
cv = KFold(n_splits=8,shuffle=True,random_state=42)



scores = cross_val_score(estimator=hm_xgb_ro.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


[15:46:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:46:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:46:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:46:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [451]:
scores

array([0.28333333, 0.23333333, 0.33333333, 0.31666667, 0.26666667,
       0.23333333, 0.28333333, 0.25      ])

In [452]:
mean_score

0.27499999999999997

In [453]:
# other metrics + graphs




#### *leave-one-run-out-shuffled* cross validation

##### Training

In [454]:
hm_xgb_ros = hyper_model(model_name='xg_boost',
                            validation_approach = 'leave-one-run-out-shuffled',
                            X=X,
                            y=y)

In [455]:
hm_xgb_ros.fit()

[15:47:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:47:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:47:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:47:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[15:48:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:49:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:49:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:49:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:  3.8min finished


[15:52:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Random HP Search Best Model Score : 0.33333333333333337
Improvement of 21.21212121212124% relative to the initial model


In [456]:
hm_xgb_ros.all_models_perf

,Model Iteration,Score


##### Test

In [457]:
# leave-one-task-out test
cv = KFold(n_splits=4,shuffle=False)

scores = cross_val_score(estimator=hm_xgb_ros.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)



[15:52:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:53:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:53:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:53:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [458]:
scores

array([0.56666667, 0.39166667, 0.49166667, 0.45833333])

In [459]:
mean_score

0.4770833333333333

In [460]:
# leave-one-task-out-shuffed test
cv = KFold(n_splits=4,shuffle=True,random_state=42)

scores = cross_val_score(estimator=hm_xgb_ros.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


[15:53:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:53:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:53:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:54:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [461]:
scores

array([0.29166667, 0.28333333, 0.29166667, 0.29166667])

In [462]:
mean_score

0.28958333333333336

In [463]:
# leave-one-run-out test
cv = KFold(n_splits=8,shuffle=False)

scores = cross_val_score(estimator=hm_xgb_ros.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


[15:54:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:54:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:54:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:54:44] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [464]:
scores

array([0.46666667, 0.48333333, 0.33333333, 0.33333333, 0.53333333,
       0.56666667, 0.5       , 0.51666667])

In [465]:
mean_score

0.4666666666666667

In [466]:
# leave-one-run-out-shuffed test
cv = KFold(n_splits=8,shuffle=True,random_state=42)



scores = cross_val_score(estimator=hm_xgb_ros.stock_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


[15:55:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:55:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:56:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:56:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [467]:
scores

array([0.28333333, 0.23333333, 0.33333333, 0.31666667, 0.26666667,
       0.23333333, 0.28333333, 0.25      ])

In [468]:
mean_score

0.27499999999999997

In [469]:
# other metrics + graphs




### Hyper_optimized model

### Stock Model

#### *leave-one-task-out* cross validation

##### Training

In [470]:
hm_xgb_to = hyper_model(model_name='xg_boost',
                            validation_approach = 'leave-one-task-out',
                            X=X,
                            y=y)

In [471]:
hm_xgb_to.fit()

[15:57:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:57:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:57:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:57:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[15:57:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:58:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:58:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:58:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  1.8min finished


[15:59:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Random HP Search Best Model Score : 0.475
Improvement of -0.4366812227074221% relative to the initial model


In [472]:
hm_xgb_to.all_models_perf

,Model Iteration,Score


##### Test

In [473]:
# leave-one-task-out test
cv = KFold(n_splits=4,shuffle=False)

scores = cross_val_score(estimator=hm_xgb_to.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)



[16:00:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:00:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:00:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:00:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [474]:
scores

array([0.55833333, 0.4       , 0.53333333, 0.40833333])

In [475]:
mean_score

0.475

In [476]:
# leave-one-task-out-shuffed test
cv = KFold(n_splits=4,shuffle=True,random_state=42)

scores = cross_val_score(estimator=hm_xgb_to.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


[16:00:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:01:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:01:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:01:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [477]:
scores

array([0.3       , 0.33333333, 0.28333333, 0.325     ])

In [478]:
mean_score

0.3104166666666667

In [479]:
# leave-one-run-out test
cv = KFold(n_splits=8,shuffle=False)

scores = cross_val_score(estimator=hm_xgb_to.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


[16:01:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:01:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:01:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:02:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [480]:
scores

array([0.5       , 0.51666667, 0.41666667, 0.46666667, 0.5       ,
       0.38333333, 0.58333333, 0.56666667])

In [481]:
mean_score

0.49166666666666664

In [482]:
# leave-one-run-out-shuffed test
cv = KFold(n_splits=8,shuffle=True,random_state=42)



scores = cross_val_score(estimator=hm_xgb_to.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


[16:03:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:03:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:03:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:03:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [483]:
scores

array([0.21666667, 0.3       , 0.21666667, 0.36666667, 0.31666667,
       0.2       , 0.4       , 0.33333333])

In [484]:
mean_score

0.29374999999999996

In [485]:
# other metrics + graphs




#### *leave-one-task-out-shuffled* cross validation

##### Training

In [486]:
hm_xgb_tos = hyper_model(model_name='xg_boost',
                            validation_approach = 'leave-one-task-out-shuffled',
                            X=X,
                            y=y)

In [487]:
hm_xgb_tos.fit()

[16:04:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:04:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:05:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:05:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[16:05:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:05:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:05:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:05:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  1.5min finished


[16:06:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Random HP Search Best Model Score : 0.31666666666666665
Improvement of 9.352517985611495% relative to the initial model


In [488]:
hm_xgb_tos.all_models_perf

,Model Iteration,Score


##### Test

In [489]:
# leave-one-task-out test
cv = KFold(n_splits=4,shuffle=False)

scores = cross_val_score(estimator=hm_xgb_tos.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)



[16:07:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:07:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:07:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:07:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [490]:
scores

array([0.58333333, 0.34166667, 0.5       , 0.39166667])

In [491]:
mean_score

0.45416666666666666

In [492]:
# leave-one-task-out-shuffed test
cv = KFold(n_splits=4,shuffle=True,random_state=42)

scores = cross_val_score(estimator=hm_xgb_tos.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


[16:08:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:08:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:08:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:08:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [493]:
scores

array([0.33333333, 0.3       , 0.31666667, 0.31666667])

In [494]:
mean_score

0.31666666666666665

In [495]:
# leave-one-run-out test
cv = KFold(n_splits=8,shuffle=False)

scores = cross_val_score(estimator=hm_xgb_tos.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


[16:08:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:09:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:09:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:09:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [496]:
scores

array([0.5       , 0.48333333, 0.41666667, 0.56666667, 0.46666667,
       0.43333333, 0.55      , 0.46666667])

In [497]:
mean_score

0.48541666666666666

In [498]:
# leave-one-run-out-shuffed test
cv = KFold(n_splits=8,shuffle=True,random_state=42)



scores = cross_val_score(estimator=hm_xgb_tos.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


[16:10:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:11:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:11:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:11:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [499]:
scores

array([0.26666667, 0.38333333, 0.31666667, 0.38333333, 0.38333333,
       0.26666667, 0.35      , 0.31666667])

In [500]:
mean_score

0.33333333333333337

In [501]:
# other metrics + graphs




#### *leave-one-run-out* cross validation

##### Training

In [502]:
hm_xgb_ro = hyper_model(model_name='xg_boost',
                            validation_approach = 'leave-one-run-out',
                            X=X,
                            y=y)

In [503]:
hm_xgb_ro.fit()

[16:13:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:13:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:13:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:13:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[16:14:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:14:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:14:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:14:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:  3.2min finished


[16:17:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Random HP Search Best Model Score : 0.49166666666666664
Improvement of 5.35714285714285% relative to the initial model


In [504]:
hm_xgb_ro.all_models_perf

,Model Iteration,Score


##### Test

In [505]:
# leave-one-task-out test
cv = KFold(n_splits=4,shuffle=False)

scores = cross_val_score(estimator=hm_xgb_ro.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)



[16:17:50] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:18:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:18:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:18:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [506]:
scores

array([0.55833333, 0.4       , 0.53333333, 0.40833333])

In [507]:
mean_score

0.475

In [508]:
# leave-one-task-out-shuffed test
cv = KFold(n_splits=4,shuffle=True,random_state=42)

scores = cross_val_score(estimator=hm_xgb_ro.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


[16:18:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:18:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:18:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:18:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [509]:
scores

array([0.3       , 0.33333333, 0.28333333, 0.325     ])

In [510]:
mean_score

0.3104166666666667

In [511]:
# leave-one-run-out test
cv = KFold(n_splits=8,shuffle=False)

scores = cross_val_score(estimator=hm_xgb_ro.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


[16:19:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:19:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:19:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:19:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [512]:
scores

array([0.5       , 0.51666667, 0.41666667, 0.46666667, 0.5       ,
       0.38333333, 0.58333333, 0.56666667])

In [513]:
mean_score

0.49166666666666664

In [514]:
# leave-one-run-out-shuffed test
cv = KFold(n_splits=8,shuffle=True,random_state=42)



scores = cross_val_score(estimator=hm_xgb_ro.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


[16:20:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:20:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:20:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:21:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [515]:
scores

array([0.21666667, 0.3       , 0.21666667, 0.36666667, 0.31666667,
       0.2       , 0.4       , 0.33333333])

In [516]:
mean_score

0.29374999999999996

In [517]:
# other metrics + graphs




#### *leave-one-run-out-shuffled* cross validation

##### Training

In [518]:
hm_xgb_ros = hyper_model(model_name='xg_boost',
                            validation_approach = 'leave-one-run-out-shuffled',
                            X=X,
                            y=y)

In [519]:
hm_xgb_ros.fit()

[16:22:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:22:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:22:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:22:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[16:23:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:23:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:24:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:24:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:  3.7min finished


[16:27:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Random HP Search Best Model Score : 0.33333333333333337
Improvement of 21.21212121212124% relative to the initial model


In [520]:
hm_xgb_ros.all_models_perf

,Model Iteration,Score


##### Test

In [521]:
# leave-one-task-out test
cv = KFold(n_splits=4,shuffle=False)

scores = cross_val_score(estimator=hm_xgb_ros.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)



[16:27:44] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:27:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:28:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:28:24] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [522]:
scores

array([0.58333333, 0.34166667, 0.5       , 0.39166667])

In [523]:
mean_score

0.45416666666666666

In [524]:
# leave-one-task-out-shuffed test
cv = KFold(n_splits=4,shuffle=True,random_state=42)

scores = cross_val_score(estimator=hm_xgb_ros.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


[16:28:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:28:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:29:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:29:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [525]:
scores

array([0.33333333, 0.3       , 0.31666667, 0.31666667])

In [526]:
mean_score

0.31666666666666665

In [527]:
# leave-one-run-out test
cv = KFold(n_splits=8,shuffle=False)

scores = cross_val_score(estimator=hm_xgb_ros.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


[16:29:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:29:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:30:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:30:24] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [528]:
scores

array([0.5       , 0.48333333, 0.41666667, 0.56666667, 0.46666667,
       0.43333333, 0.55      , 0.46666667])

In [529]:
mean_score

0.48541666666666666

In [530]:
# leave-one-run-out-shuffed test
cv = KFold(n_splits=8,shuffle=True,random_state=42)



scores = cross_val_score(estimator=hm_xgb_ros.rd_model,cv=cv,X=X,y=y)

mean_score = np.mean(scores)


[16:31:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:31:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:31:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:32:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [531]:
scores

array([0.26666667, 0.38333333, 0.31666667, 0.38333333, 0.38333333,
       0.26666667, 0.35      , 0.31666667])

In [532]:
mean_score

0.33333333333333337

In [533]:
# other metrics + graphs


